In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from analysis.plotutility import *
from config.selectionconfig import plotsetting as pltcfg

In [3]:
import os

In [4]:
vis = Visualizer(pltcfg)

In [5]:
vis.grepweights(output=True)

In [6]:
vis.indir

'/uscms/home/joyzhou/nobackup/output'

In [8]:
# test combine_cf()
dirname = pjoin(vis.indir, 'DYJets')
raw_cutflow = vis.combine_cf(dirname, 'DYJetsToLL_M-10to50', output=True)
raw_cutflow

,DYJetsToLL_M-10to50
initial,22445405
ElectronSelection,22314144
MuonSelection,21925141
TauSelection,277


In [9]:
# test compute_allcf()
raw_df, wgt_df = vis.compute_allcf()

In [10]:
raw_df

,DYJetsToLL_M-10to50,DYJetsToLL_M-50,ZH_HToBB_ZToLL_M-125,ZH_HToBB_ZToQQ_M-125
initial,22445405,92312235,2062940,4092774
ElectronSelection,22314144,68892058,1236115,3577235
MuonSelection,21925141,47130535,609499,3373217
TauSelection,277,72801,11019,9159


In [11]:
wgt_df

,DYJetsToLL_M-10to50,DYJetsToLL_M-50,ZH_HToBB_ZToLL_M-125,ZH_HToBB_ZToQQ_M-125
initial,654524.197717,301299.553562,15.371238,14.371148
ElectronSelection,650696.532290,224858.019301,9.210456,12.560912
MuonSelection,639352.924256,153830.195473,4.541458,11.844534
TauSelection,8.077520,237.616485,0.082104,0.032160
